# Fire Agent with LlamaIndex

## Install Dependencies

In [1]:
# !pip install uv
# !uv pip install --system -qU llama-index==0.11.6 llama-index-llms-openai llama-index-readers-file llama-index-embeddings-openai llama-index-llms-openai-like "openinference-instrumentation-llama-index>=2" arize-phoenix python-dotenv

  Using cached uv-0.4.27-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
Using cached uv-0.4.27-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.8 MB)


## Setup API Keys
To run the rest of the notebook you will need access to an OctoAI API key. You can sign up for an account [here](https://octoai.cloud/). If you need further guidance you can check OctoAI's [documentation page](https://octo.ai/docs/getting-started/how-to-create-octoai-access-token).

In [1]:
from os import environ
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = environ["OPENAI_API_KEY"]

## Import libraries and setup LlamaIndex

In [2]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI


# Create an llm object to use for the QueryEngine and the ReActAgent
llm = OpenAI(model="gpt-4")

# Set up Phoenix

In [6]:
import phoenix as px
session = px.launch_app()

/home/hongeinh/anaconda3/envs/hackathon/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [7]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from phoenix.otel import register

tracer_provider = register()
LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



## Load Documents

In [8]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/nfpa"
    )
    nfpa_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

This is the point we create our vector indexes, by calculating the embedding vectors for each of the chunks. You only need to run this once.

In [9]:
if not index_loaded:
    # load data
    nfpa_docs = SimpleDirectoryReader(
        input_files=["./NFPA10-2022.pdf"]
    ).load_data()

    # build index
    nfpa_index = VectorStoreIndex.from_documents(nfpa_docs, show_progress=True)

    # persist index
    nfpa_index.storage_context.persist(persist_dir="./storage/nfpa")

Now create the query engines.

In [10]:
nfpa_engine = nfpa_index.as_query_engine(similarity_top_k=3, llm=llm)

We can now define the query engines as tools that will be used by the agent.

As there is a query engine per document we need to also define one tool for each of them.

In [11]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=nfpa_engine,
        metadata=ToolMetadata(
            name="NFPA",
            description=(
                "Provides information about Fire regulations for year 2022. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    )
]

## Creating the Agent
Now we have all the elements to create a LlamaIndex ReactAgent

In [12]:
agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    max_turns=10,
)

In [14]:
# query = "What would a D Class fire extinguisher be used for?"
# response = agent.chat(query)
# print(str(response))

> Running step ba3263fa-8c81-401e-aebd-bbfade5de226. Step input: What would a D Class fire extinguisher be used for?
Thought: The user is asking about the use of a D Class fire extinguisher. I can use the NFPA tool to get the specific information about this type of fire extinguisher.
Action: NFPA
Action Input: {'input': 'What is a D Class fire extinguisher used for?'}
Observation: A D Class fire extinguisher is used for fires involving various forms of combustible metals such as powders, flakes, shavings, chips, or liquid states that burn at extremely high temperatures. These fires are capable of breaking down normal extinguishing agents, causing undesirable reactions. Therefore, only extinguishing agents specifically tested and listed for use on particular combustible Class D metal fire hazards should be selected and provided.
> Running step 5ac540da-e190-42dd-8a9e-ae5e9e7a36b1. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer

# Set up retrieval

In [47]:
import re

In [48]:
def retrieve_relevant_section(query, response_text, nfpa_index):
    """
    This function retrieves the most relevant section from the NFPA document
    based on the agent's response.

    Args:
        query (str): The user's original query.
        response_text (str): The text content of the agent's response.
        nfpa_index (VectorStoreIndex): The vector store index containing the NFPA document.

    Returns:
        tuple: A tuple containing the page number and section number, or None if not found.
    """

    # Combine the query and response to create a more focused search query
    search_query = f"{query} {response_text}"

    # Use query engine to retrieve documents (sections)
    query_engine = nfpa_index.as_retriever()
    search_results = query_engine.retrieve(search_query)

    # Sort results by similarity (assuming a "score" field in response)
    sorted_results = sorted(search_results, key=lambda result: result.score, reverse=True)

    # Extract the title of the most relevant section (top result)
    if sorted_results:
        top_result = sorted_results[0]
        relevant_node = top_result.node
        metadata = relevant_node.metadata

        page_number = metadata['page_label']

        # Extract section number from the text content
        section_text = relevant_node.text
        # Use regular expressions or other text processing techniques to extract the section number
        # This will depend on the specific format of your document and the way section numbers are represented
        # For example, if section numbers are at the beginning of paragraphs:
        section_match = re.search(r"^\s*(\w+\.\d+\.\d+)", section_text, re.MULTILINE)
        if section_match:
            section_number = section_match.group(1)
        else:
            section_number = "N/A"  # Or handle the case where section number cannot be extracted

        return page_number, section_number
    else:
        return None, None

Now we can interact with the agent and ask a question.

In [49]:
def get_agent_response_with_section(query):
    response = agent.chat(query)

    # Assuming "text" is the attribute containing the response content
    response_text = response.response

    # Retrieve the relevant section from the PDF
    page, section = retrieve_relevant_section(query, response_text, nfpa_index)

    if section:
        return f"{response_text}\n\n**Reference:** Section {section} of NFPA 10-2022 on page {page}"
    else:
        return response_text

In [50]:
query = "What would a D Class fire extinguisher be used for?"
agent_response = get_agent_response_with_section(query)
print(agent_response)

> Running step 129eeefc-61e2-4bb1-af47-7d480108546e. Step input: What would a D Class fire extinguisher be used for?
Thought: The user is asking about the use of a D Class fire extinguisher. I can use the NFPA tool to provide the most accurate and up-to-date information.
Action: NFPA
Action Input: {'input': 'What is a D Class fire extinguisher used for?'}
Observation: A Class D fire extinguisher is used for fires involving combustible metals such as powders, flakes, shavings, chips, or liquid states that burn at extremely high temperatures. These fires can break down normal extinguishing agents and cause undesirable reactions, so only extinguishing agents specifically tested and listed for use on particular combustible Class D metal fire hazards should be used. The selection of fire extinguishers for these hazards should be based on equipment manufacturers' recommendations.
> Running step fdcafe7e-e931-48e9-9e01-d23434654d6b. Step input: None
Thought: I can answer without using any mor

In [18]:
# !pip install dill


I0000 00:00:1729967644.958448   36077 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


In [20]:
import dill

with open('react_agent.dill', 'wb') as f:
    dill.dump(agent, f)
